In [1]:
#@title Install prerequsite
!pip install GEOparse

     |████████████████████████████████| 278 kB 8.3 MB/s 
  Created wheel for GEOparse: filename=GEOparse-2.0.3-py3-none-any.whl size=29063 sha256=5839c02ceb857e74bf7f541d332629bff3525d15be1280a4af8dd507d3dbb76e
  Stored in directory: /root/.cache/pip/wheels/4d/15/e8/fbf3b47444215d9728c20d7b35436b50086aa67c2ad6dcedad
Successfully built GEOparse


In [2]:
#@title Mount Google Drive (You don't need to run this if you are running notebooks on your laptop)

from google.colab import drive

# The following command will prompt a URL for you to click and obtain the
# authorization code

drive.mount("/content/drive")

Mounted at /content/drive


This code is an attempt to reproduce the analysis preformed in the following paper: ["Upper airway gene expression reveals suppressed immune responses to SARS-CoV-2 compared with other respiratory viruses"](https://pubmed.ncbi.nlm.nih.gov/33203890/) by Mick, Kamm, et al. published in *Nature Communications*. 

First we need to download the data from the NCBI Database [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE156063).

Download the following zip files and move them into your drive: [GSE156063_swab_gene_counts.csv.gz](https://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nnn/GSE156063/suppl/GSE156063_swab_gene_counts.csv.gz) (count matrix) and [GSE156063_family.soft.gz](https://ftp.ncbi.nlm.nih.gov/geo/series/GSE156nnn/GSE156063/soft/GSE156063_family.soft.gz) (metadata).

In [3]:
# Set up data folder
from pathlib import Path

DATA = Path("/content/drive/My Drive/Geonomics_E4060/data/project")

Unzip and load the metadata, we will have to use GEOParse.

In [4]:
# Read in the metadata and preprocess into a dataframe
import GEOparse

metadata_file = DATA / "GSE156063_family.soft.gz"

gse = GEOparse.get_GEO(filepath=str(metadata_file))

20-Dec-2021 22:51:08 INFO GEOparse - Parsing /content/drive/My Drive/Geonomics_E4060/data/project/GSE156063_family.soft.gz: 
20-Dec-2021 22:51:08 DEBUG GEOparse - DATABASE: GeoMiame
20-Dec-2021 22:51:08 DEBUG GEOparse - SERIES: GSE156063
20-Dec-2021 22:51:08 DEBUG GEOparse - PLATFORM: GPL24676
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721578
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721579
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721580
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721581
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721582
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721583
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721584
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721585
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721586
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721587
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721588
20-Dec-2021 22:51:09 DEBUG GEOparse - SAMPLE: GSM4721589
20-Dec-2021 22:51:09 

We need to preprocess the metadata: for each sample we want the following columns: idseq_sample_name (name of matching IDSeq sample (used to map to count data)), gender (gender of patient), age (age of patient), sc2_pcr (result of SARS-COV-2 PCR test, this determines whether the patient is in the COVID group or not), sc2_rpm (reads-per-million mapping to SARS-COV-2 metagenomic sequencing data), and viral_status (either SC2 (for SARS-CoV-2), other_virus or no_virus).

In [5]:
gse.gsms['GSM4721578'].metadata

{'channel_count': ['1'],
 'characteristics_ch1': ['gender: F',
  'age: 62',
  'sars-cov-2 rpm: NEG',
  'sars-cov-2 pcr: 0.313853190805252',
  'disease state: no virus'],
 'contact_address': ['499 Illinois St'],
 'contact_city': ['San Francisco'],
 'contact_country': ['USA'],
 'contact_department': ['Medicine'],
 'contact_email': ['chaz.langelier@ucsf.edu, chazlangelier@gmail.com'],
 'contact_institute': ['University of California San Francisco'],
 'contact_name': ['Charles,,Langelier'],
 'contact_phone': ['801-201-5049'],
 'contact_state': ['CA'],
 'contact_zip/postal_code': ['94158'],
 'data_processing': ['Alignment against human genome- Kallisto v 0.46.1',
  'Gene-level counts were generated from the transcript-level abundance estimates using the R package tximport',
  'Fitering to retain samples with 400,000 gene counts',
  'Differential expression with R package limma',
  'Genome_build: HG-38, ENSEMBL v. 99',
  'Supplementary_files_format_and_content: .csv files with genecounts'],


In [6]:
import pandas as pd

# Columns we want in our metadata DataFrame:
columns = ['idseq_sample_name', 'gender', 'age', 'sc2_pcr', 'sc2_rpm', 'viral_status']

# From the above example of the metadata from a sample, we find that the 'title' key maps to 
# the 'idseq_sample_name' and the 'characteristics_ch1' key maps to a list that has a dictionary-type 
# structure, we want all 5 of these values: 'age', 'gender', 'sars-cov-2 rpm' which actually should be
# 'sc2_pcr', 'sars-cov-2 pcr' which actually should be 'sc2_rpm', and viral_status.
ch_column = 'characteristics_ch1'
characteristics = ['gender', 'age', 'sars-cov-2 rpm', 'sars-cov-2 pcr', 'disease state']

# Create list of titles and characteristic dictionary to later be combined and converted into a Dataframe
titles = []
ch_dict = {}
for ch in characteristics:
    ch_dict.update({ch : []})

# Iterate through the 234 samples and construct the metadata DataFrame
for gsm in gse.gsms.keys():
    titles.append(gse.gsms[gsm].metadata['title'][0])
    for ch in gse.gsms[gsm].metadata[ch_column]:
        x = ch.split(': ')
        ch_dict[x[0]].append("_".join(x[1].split(" "))) # want to remove the space in viral_status

# Create metadata dictionary, copy in titles & ch_dict, and convert it to DataFrame
metadata = {}
metadata['idseq_sample_name'] = titles
for i in range(len(characteristics)):
    metadata[columns[i+1]] = ch_dict[characteristics[i]]
metadata = pd.DataFrame.from_dict(metadata)
metadata.head()

,idseq_sample_name,gender,age,sc2_pcr,sc2_rpm,viral_status
0,RR057e_00202_N05_S78,F,62,NEG,0.313853190805252,no_virus
1,RR057e_00080_H20_S312,M,81,NEG,0.098334479421691,no_virus
2,RR057e_00287_L09_S140,F,76,NEG,0.914830871073219,no_virus
3,RR057e_00753_G10_S151,F,36,POS,403.657304115618,SC2
4,RR057e_00751_C10_S147,F,58,POS,96314.7658717924,SC2


Save the preprocessed metadata so that it can be used in the DE analysis (see `DE_Analysis.Rmd`).

In [7]:
# Write metadata to .txt file
output_metadata_file = DATA / "GSE156063_metadata.txt"

metadata.to_csv(output_metadata_file, sep='\t', index=False)